In [3]:
!pip install --quiet simpletransformers -qq

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 

In [4]:
import pandas as pd
import json
import time
import warnings
warnings.filterwarnings("ignore")

In [5]:
from simpletransformers.t5 import T5Model

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing data and cleaning**


Input to the model:

1.   Input text-> question
2.   traget text ->answers
3.   context ->context
4.   prefix -> question








In [7]:
temp = pd.read_csv("/content/drive/MyDrive/NLP/openai/QASPER/QASPER_openAI.csv")
temp_train= pd.DataFrame()
temp=temp.replace(['Answer 1:',],'', regex=True)
temp=temp.replace(['Question 1:',],'', regex=True)
temp_train["input_text"] = temp['question']
temp_train["target_text"] = temp['answer']
temp_train["context"] = temp['context']
temp_train["prefix"] = "question:"

In [8]:
temp_train.shape

(20153, 4)

In [16]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 4,
    "num_train_epochs": 2,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "fp16": False,
}
model2 = T5Model("t5", "t5-base", args=model_args)

In [18]:
model2.train_model(temp_train)

  0%|          | 0/20153 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/5039 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/5039 [00:00<?, ?it/s]

(10078, 0.6598505061264419)

In [15]:
context = 'We fine tune a pre-trained BERT embedding to perform transfer learning for our instance selection task. We use a 12-layer implementation which has been shown to generalize and perform well when fine-tuned for new tasks such as question answering, text classification, and named entity recognition. To apply the model to our task, we add an additional dense layer to the BERT architecture with 10% dropout, mapping the last pooled layer to the object classes in our data.'
question ='How was the model fine tuned?'

input_str = f'context: {context}  question: {question}'

outputs = model2.predict([input_str])
answer = outputs[0]

print(answer)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

The last layer of our data is fine tuned to include a single layer of entity recognition,


saving the model:

In [45]:
import pickle

pickle.dump(model2,open(r"/content/drive/MyDrive/NLP/T5_answer_model.pkl","wb"))